In [71]:
import glob
import json
import os

import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from ipywidgets import fixed, interactive

In [72]:
def get_subjects(folder):
    subjects = glob.glob(os.path.join(folder, 's*'))
    subjects = [os.path.basename(x) for x in subjects]
    subjects.sort()

    return subjects

In [73]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    return result

In [74]:
def path_nii(subject, isMask):
    suffix_image = "_rec-mial_T2w.nii.gz"
    suffix_mask = "_rec-mial_dseg.nii.gz"
    
    if isMask == False:
        return os.path.join(folder, 
                              subject, 
                              "anat", 
                              subject+suffix_image)
    else:
        return os.path.join(folder, 
                              subject, 
                              "anat", 
                              subject+suffix_mask)

In [75]:
def draw_image(index, sub_name, isMask, orientation):
    nii_im = nib.load(path_nii(sub_name, isMask))
    im = nii_im.get_fdata()
    
    fig = plt.figure(figsize=(8, 8))
    if orientation == "coronal":
        im = rotate_image(im, 90)
        plt.imshow(im[:, index, :])
    elif orientation == "axial":
        im = rotate_image(im, 90)
        plt.imshow(im[:, :, index])
    else:
        im = rotate_image(im, 90)
        plt.imshow(im[index, :, :])
    plt.show()
    

def show_image(subject, isMask):
    interactive_plot = interactive(draw_image, index=(0, 255),
                                   sub_name = fixed(subject),
                                   isMask=fixed(isMask),
                                   orientation=["axial", "coronal", "sagittal"])
    
    output = interactive_plot.children[-1]
    display(interactive_plot)

In [76]:
folder = 'feta_2.1/'

subjects = get_subjects(folder)

In [77]:
show_image(subjects[0], isMask = True)

interactive(children=(IntSlider(value=127, description='index', max=255), Dropdown(description='orientation', …